In [1]:
from PrismFileReader import PrismFileReader
from PrismModule import PrismModule
import random
import os

def diffLists(l1,l2):
    l1s = set(l1)
    l2s = set(l2)
    res = l1s.symmetric_difference(l2s)
    return list(res)

def sample(l,n):
    return random.sample(l,n)

def createStateString(variable,s):
    ss = '('+variable+'='+str(s)+')'
    return ss

def pickStates(previousStatesList,numToPick,currentStatesList):
    statesPicked = []
    n = numToPick
    if previousStatesList is not None:
        statesPicked = previousStatesList[0:n]
        n = n - len(statesPicked)
    if n> 0:
        statesToPickFrom = diffLists(statesPicked,currentStatesList)
        newStatesPicked = sample(statesToPickFrom,n)
        statesPicked = statesPicked + newStatesPicked
    return statesPicked

def ensureDir(dirLoc):
    if not os.path.exists(dirLoc):
        os.makedirs(dirLoc)

def getTermWidth():
    rows, columns = os.popen('stty size', 'r').read().split()
    width = int(columns)
    return width

def printC(string,width):
    #width = os.get_terminal_size().columns
    
    
    print(string.center(width))

    
def generatePrismFiles(sVar,fpath,fn,numAgents,numGoals,numFailStates,numDoorStates,numDoors,saveLoc,previousInitStates,previousGoalStates,previousFailStates,previousDoorStates):
    if numDoors != numDoorStates:
        print "error"
        return 
    resDir = saveLoc+fn+"/"+"r"+str(numAgents)+"/t"+str(numGoals)+"/fs"+str(numFailStates)+"/d"+str(numDoors)+"/"
    saveresdir = resDir +"results/"
    ensureDir(saveresdir)
    
    pfr = PrismFileReader(fpath+fn+".prism")

    #assumption there is no door
    #add door variables
    pfr.addConstant('unknown','int','-1')
    pfr.addConstant('open','int','1')
    pfr.addConstant('closed','int','0')

    #check if it has a failstate
    #otherwise add it
    failstate = 'failstate'
    if not failstate in pfr.constants:
        pfr.addConstant(failstate,'int','-1')

    successProbConstant = 'p'
    succProb = 0.8
    if not successProbConstant in pfr.constants:
        pfr.addConstant(successProbConstant,'double',str(succProb))
        

    #assumption the module is called world
    moduleName = pfr.moduleNames[0]
    print "Module"
    print moduleName

    #get the first variable
    variables = pfr.getVariables(moduleName)

    #cleaning up doors
    doorVars = []
    #does the model have any 'doors' before hand
    for v in variables:
        if 'door' in v:
            doorVars.append(v)
            
            #raise Exception("Door in this model")
    for doorVar in doorVars:
        del variables[doorVar]

    for v in variables:
        if 'door' in v:
            raise Exception("Door in this model:"+str(variables))
    #print "Variables"
    #print variables
    #just get the first one
    if sVar is not None and sVar in variables:
        variable = v
    else:
        #pick the variable with the greatest range
        maxRange = 0
        
        for v in variables:
            vRange = variables[v].getRangeValues()
            varRange = vRange[1]-vRange[0]
            if(varRange > maxRange):
                maxRange = varRange
                variable =v 
            

    print "Variable"
    print variable

    #lets get its range
    varRange = variables[variable].getRangeValues()
    #if range doesnt have a failstate we'll have to modify this variable
    if varRange[0] != -1:
        pfr.changeVariableMinRange(variable,pfr.constants[failstate])
        varRange = variables[variable].getRangeValues()

    varRangeVals = range(varRange[0]+1,varRange[1])

    numStates = len(varRangeVals)


    #generate the states we need
    #initial positions
    #print "Generating states of concern"
    if len(previousDoorStates)>=numDoors*2:
        doorStates = previousDoorStates[0:numDoors*2]
    else:
        previousDoors = len(previousDoorStates)/2
        numDoorsToAdd = numDoors - previousDoors
        #creating previous state pairs
        statePairs = []
        for i in range(0,len(previousDoorStates),2):
            statePairs.append([previousDoorStates[i],previousDoorStates[i+1]])
            
        doorStates= pfr.pickActions(moduleName,variable,numDoorsToAdd,statePairs)
        doorStates = previousDoorStates + doorStates
        
    if(len(set(doorStates)) != len(doorStates)):
        print doorStates
        print "Door States have repeats!!"
        raise Exception("Door states have repeats ")
        #return 
        #raw_input()
    statesLeft = diffLists(varRangeVals,doorStates)
    initStates = pickStates(previousInitStates,numAgents,statesLeft)
    statesLeft = diffLists(statesLeft,initStates)
    goalStates = pickStates(previousGoalStates,numGoals,statesLeft)
    #print goalStates
    #raw_input()
    statesLeft = diffLists(statesLeft,goalStates)
    if numFailStates > numGoals:
        failStates = pickStates(previousFailStates,numFailStates-numGoals,statesLeft)
        statesLeft = diffLists(statesLeft,failStates)
        failStates = failStates + goalStates
    else:
        failStates = goalStates[0:numFailStates]
    #instead of picking doorstates randomly
    #pick two connected ones
    #we can do this by picking from actions instead
    
    #doorStates = pickStates(previousDoorStates,numDoorStates*2,statesLeft)
    #sample(statesLeft,numDoorStates*2)
    #statesLeft = diffLists(statesLeft,doorStates)
    print "Initial States"
    print initStates
    print "Goal States"
    print goalStates
    print "Fail States"
    print failStates
    print "Doors"
    print doorStates
    #raw_input()


    #add fail states
    print "Adding failure states"
    for s in failStates:
        ss = createStateString(variable,s)
        #print ss
        pfr.addFailureStateToAllActions(moduleName,ss,'1-p','p',isDest=True)
        

    for i in range(0,numDoors):
        #add door variables
    
        doorVarName = 'door'+str(i)
        #print doorVarName
        pfr.addVariable(moduleName,doorVarName,pfr.constants['unknown'],pfr.constants['open'],pfr.constants['unknown'],pfr.constants['unknown'])

    numDoorVar = 0
    for i in range(0,len(doorStates),2):
        #print "counted doors"
        #print numDoorVar
        #print "total doors"
        #print numDoors
        if(numDoorVar == numDoors):
            break
        doorVarName = 'door'+str(numDoorVar)
        s1 = doorStates[i]
        s2 = doorStates[i+1]
        #print doorVarName
        #print s1
        #print s2

        #raw_input()
        s1s = createStateString(variable,s1)
        s2s = createStateString(variable,s2)
        pfr.addDoor(moduleName,doorVarName,s1s,s2s,'p','1-p')
        numDoorVar = numDoorVar+1

    ensureDir(resDir)
    pfr.writeGoalStates(goalStates,variable,resDir+fn+'.props')
    pfr.writeGoalStatesSplit(goalStates,variable,resDir+fn+'.prop')

    #now lets do stuff
    for i in range(numAgents):
        
        pfr.changeVariableInitValue(moduleName,variable,initStates[i])
        
        lines = pfr.createFileLinesToWrite()
        pfr.writeLinesToFile(lines,resDir+fn+str(i)+'.prism')

    #print "Initial States"
    #print initStates
    #print "Goal States"
    #print goalStates
    #print "Fail States"
    #print failStates
    #print "Doors"
    #print doorStates
    print "File Loc "+resDir
    #raw_input()
    pfr.cleanModules()
    return (initStates,goalStates,failStates,doorStates)





In [2]:
pfr = PrismFileReader(None)

an empty file


In [3]:
#add a constant 
pfr.addConstant('p','double','0.8')
pfr.moduleNames = ['world']
mod = PrismModule('world',[],pfr.constants)


Initializing Module world


In [4]:
mod.addVariable('s',-1,10,0,0)

In [5]:
mod.variables

{'s': s:0(0) range:[-1,10]}

In [6]:
pa = mod.createAction('s1_s2',{'"time"':1.0},'(s=0)',['(s=1)','(s=2)'],['p','1-p'])

In [7]:
mod.actions.append(pa)
pfr.modVars = [mod]
pfr.addLabel('v0','(s=0)',mod.variables)
pfr.addLabel('v1','(s=1)',mod.variables)
pfr.rewardNames=['"time"']

In [8]:
lines = pfr.createFileLinesToWrite()
pfr.writeLinesToFile(lines,'grid.prism')

In [9]:
#do a grid 
gpfr = PrismFileReader(None)
gpfr.addConstant('p','double','0.8')
gpfr.addConstant('failstate','int','-1')
modName = 'grid'
gpfr.moduleNames=[modName]
gmod = PrismModule(modName,[],gpfr.constants)
xside = 3 
yside = 3
gridLim = xside*yside
varname = 's'
gmod.addVariable(varname,gpfr.constants['failstate'],gridLim,0,0)
gpfr.rewardNames=['"time"']
gpfr.modVars = [gmod]

an empty file
Initializing Module grid


In [10]:
def linInd(x,y,xside,yside):
    return y*yside + x

def stateString(varname,varvalue):
    return '('+varname+'='+str(varvalue)+')'

def stateLabel(varname,varvalue):
    return varname+str(varvalue)

In [11]:
scount = 0 
for x in range(xside):
    for y in range(yside):
        scount = linInd(x,y,xside,yside)
        svar = stateLabel(varname,scount)
        sstr = stateString(varname,scount)
        if gpfr.labels is None:
            gpfr.addLabel(svar,sstr,gmod.variables)
        if not svar in gpfr.labels:
            gpfr.addLabel(svar,sstr,gmod.variables)
        xx = min(x+1,xside-1)
        yy = min(y+1,yside-1)
        _x = max(x-1,0)
        _y = max(y-1,0)
        combs = [(x,yy),(x,_y),(xx,y),(_x,y)]
        
        for xy in combs:
            xp = xy[0]
            yp = xy[1]
            if(xp == x and yp == y):
                continue
            pcount = linInd(xp,yp,xside,yside)
            pstr = stateString(varname,pcount)
            pvar = stateLabel(varname,pcount)
            if not pvar in gpfr.labels:
                gpfr.addLabel(pvar,pstr,gmod.variables)
            #pa = mod.createAction('s1_s2',{'"time"':1.0},'(s=0)',['(s=1)','(s=2)'],['p','1-p'])
            pa = gmod.createAction(svar+'_'+pvar,{'"time"':1.0},sstr,[pstr,'(s=failstate)'],['p','1-p'])
            print pa
            gmod.actions.append(pa)
            

s0_s3[s:0(0) range:[failstate,9]][{'states': [s:3(0) range:[failstate,9]], 'prob': 'p'}, {'states': [s:failstate(0) range:[failstate,9]], 'prob': '1-p'}]
s0_s1[s:0(0) range:[failstate,9]][{'states': [s:1(0) range:[failstate,9]], 'prob': 'p'}, {'states': [s:failstate(0) range:[failstate,9]], 'prob': '1-p'}]
s3_s6[s:3(0) range:[failstate,9]][{'states': [s:6(0) range:[failstate,9]], 'prob': 'p'}, {'states': [s:failstate(0) range:[failstate,9]], 'prob': '1-p'}]
s3_s0[s:3(0) range:[failstate,9]][{'states': [s:0(0) range:[failstate,9]], 'prob': 'p'}, {'states': [s:failstate(0) range:[failstate,9]], 'prob': '1-p'}]
s3_s4[s:3(0) range:[failstate,9]][{'states': [s:4(0) range:[failstate,9]], 'prob': 'p'}, {'states': [s:failstate(0) range:[failstate,9]], 'prob': '1-p'}]
s6_s3[s:6(0) range:[failstate,9]][{'states': [s:3(0) range:[failstate,9]], 'prob': 'p'}, {'states': [s:failstate(0) range:[failstate,9]], 'prob': '1-p'}]
s6_s7[s:6(0) range:[failstate,9]][{'states': [s:7(0) range:[failstate,9]], '

In [12]:
lines = gpfr.createFileLinesToWrite()
gpfr.writeLinesToFile(lines,'grid.prism')